In [ ]:
import numpy as np
from langchain_community.llms import Tongyi
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Tongyi(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0.7)

In [ ]:
prompt0 = "请介绍一下微观经济学的前提基础"
llm.invoke(prompt0)

In [ ]:
llm.get_num_tokens("fasfdsafreqwrwe")  # 需要依赖torch

In [ ]:
help(llm)

In [ ]:
dir(llm)

In [ ]:
llm_di = Tongyi(temperature=0.2)

In [ ]:
s = "请帮我写一副关于龙年的对联"

In [ ]:
llm.invoke(s)

In [ ]:
llm_di.invoke(s)

In [ ]:
llm_ga = Tongyi(temperature=0.9)
res = llm_ga.generate(["写一首唐诗绝句", "写一首宋词《贺新郎》"])

In [ ]:
print(res.generations[0][0].text)
res.generations[0][0].generation_info

In [ ]:
print(res.generations[1][0].text)
res.generations[1][0].generation_info

In [ ]:
llm_ga.invoke("这首贺新郎是否符合词牌格律？")  # 显然模型调用没法识别历史提问上下文

In [ ]:
from langchain import PromptTemplate

template = """
我希望你能充当新公司的命名顾问。
一个生产{product}的公司的好名字是什么？
"""

prompt_template = PromptTemplate(
    input_variables=["product"],
    template=template,
)
prompt = prompt_template.format(product="彩色袜子")

In [ ]:
print(llm_ga.invoke(prompt))

In [ ]:
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="给我讲一个{adjective}的关于{content}的笑话。"
)
prompt = multiple_input_prompt.format(adjective="有趣的", content="小鸡")

In [ ]:
print(llm_ga.invoke(prompt))

In [ ]:
from langchain import PromptTemplate, FewShotPromptTemplate

# 首先，创建Few Shot示例列表
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

# 接下来，我们指定用于格式化示例的模板。
# 我们使用`PromptTemplate`类来实现这个目的。
example_formatter_template = """Word: {word}
Antonym: {antonym}
"""

example_prompt_template = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

# 最后，创建`FewShotPromptTemplate`对象。
few_shot_prompt = FewShotPromptTemplate(
    # 这些是我们要插入到提示中的示例。
    examples=examples,
    # 这是我们在将示例插入到提示中时要使用的格式。
    example_prompt=example_prompt_template,
    # 前缀是出现在提示中示例之前的一些文本。
    # 通常，这包括一些说明。
    prefix="Give the antonym of every input\n",
    # 后缀是出现在提示中示例之后的一些文本。
    # 通常，这是用户输入的地方。
    suffix="Word: {input}\nAntonym: ",
    # 输入变量是整个提示期望的变量。
    input_variables=["input"],
    # 示例分隔符是我们将前缀、示例和后缀连接在一起的字符串。
    example_separator="\n",
)

# 现在，我们可以使用`format`方法生成一个提示。
prompt = few_shot_prompt.format(input="big")
print(prompt)

In [ ]:
llm.invoke(few_shot_prompt.format(input="生机"))

In [ ]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage

chat_ty = ChatTongyi(
    streaming=True,
)
res = chat_ty.stream([HumanMessage(content="hi")], streaming=True)
try:
    for chunk in res:
        print(chunk)
except TypeError as e:
    print("")

In [ ]:
res = chat_ty.stream([HumanMessage(content="帮我写副对联")], streaming=True)

In [ ]:
try:
    for chunk in res:
        print(chunk.content)
except TypeError as e:
    print("")

In [ ]:
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["product"],
    template="我希望你能充当新公司的命名顾问。一个生产{product}的公司好的中文名字是什么,请给出五个选项？",
)

In [ ]:
chain = prompt | llm

In [ ]:
chain.invoke("彩色袜子")

In [ ]:
input_list = [
    {"product": "袜子"},
    {"product": "电脑"},
    {"product": "男鞋"}
]
chain.invoke(input_list)

In [ ]:
for input in input_list:
    chain.invoke(input)

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="请列出五个 {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)
prompt

In [ ]:
input0 = prompt.format(subject="中国的名山")
output = llm.invoke(input0)
res0 = output_parser.parse(output)
res0

### 其他解析器

In [ ]:
from langchain.output_parsers import DatetimeOutputParser, StructuredOutputParser
# 日期时间解析器:DatetimeOutputParser
# 结构化输出解析器:StructuredOutputParser

### 历史消息

## Memory

大多数的 LLM 应用程序都会有一个会话接口，允许我们和 LLM 进行多轮的对话，并有一定的上下文记忆能力。但实际上，模型本身是不会记忆任何上下文的，只能依靠用户本身的输入去产生输出。而实现这个记忆功能，就需要额外的模块去保存我们和模型对话的上下文信息，然后在下一次请求时，把所有的历史信息都输入给模型，让模型输出最终结果。

而在 LangChain 中，提供这个功能的模块就称为 Memory，用于存储用户和模型交互的历史信息。在 LangChain 中根据功能和返回值的不同，会有多种不同的 Memory 类型，主要可以分为以下几个类别：

* 对话缓冲区内存（ConversationBufferMemory），最基础的内存模块，用于存储历史的信息
* 对话缓冲器窗口内存（ConversationBufferWindowMemory），只保存最后的 K 轮对话的信息，因此这种内存空间使用会相对较少
* 对话摘要内存（ConversationSummaryMemory），这种模式会对历史的所有信息进行抽取，生成摘要信息，然后将摘要信息作为历史信息进行保存。
* 对话摘要缓存内存（ConversationSummaryBufferMemory），这个和上面的作用基本一致，但是有最大 token 数的限制，达到这个最大 token 数的时候就会进行合并历史信息生成摘要

值得注意的是，对话摘要内存的设计出发点就是语言模型能支持的上下文长度是有限的（一般是 2048），超过了这个长度的数据天然的就被截断了。这个类会根据对话的轮次进行合并，默认值是 2，也就是每 2 轮就开启一次调用 LLM 去合并历史信息。

In [ ]:
# 链外管理内存
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("hi!")
history.add_ai_message("whats up?")
history.messages

In [ ]:
# 对话缓冲区内存（ConversationBufferMemory）
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)
conversation.predict(input="Hi there!")

In [ ]:
conversation.predict(input="你是谁")

In [ ]:
conversation.memory.chat_memory.messages

In [ ]:
from langchain.schema import messages_from_dict, messages_to_dict
import json

dicts = messages_to_dict(conversation.memory.chat_memory.messages)
dicts

In [ ]:
import pickle

f = open("./memory", 'wb')
pickle.dump(dicts, f)
f.close()

In [ ]:
dicts_load = pickle.load(open("./memory", "rb"))
dicts_load

In [ ]:
# 向对话添加记忆
new_messages = messages_from_dict(dicts_load)
retrieved_chat_history = ChatMessageHistory(messages=new_messages)
retrieved_memory = ConversationBufferMemory(chat_memory=retrieved_chat_history)

conversation_reload = ConversationChain(
    llm=llm,
    verbose=True,
    memory=retrieved_memory
)
conversation_reload.predict(input="我回来了")

# 检索增强 与 文本向量器

## 文档导入

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain_community.llms import Tongyi

model_ty = Tongyi(temperature=0.1)

In [8]:
# 不用print函数，直接用变量名称打印输出
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader_pdf = PyPDFLoader(r"D:\workspace\收发文件\新型冠状病毒肺炎防控方案（第八版）.pdf")
docs_pdf = loader_pdf.load()
type(docs_pdf)
len(docs_pdf)
type(docs_pdf[0])

In [ ]:
docs_pdf[0].metadata
docs_pdf[0].page_content[0:100]

In [ ]:
from langchain.document_loaders import TextLoader

loader_txt = TextLoader(r'D:\workspace\收发文件\李卅移交资料.txt', encoding='utf8')
docs_txt = loader_txt.load()
len(docs_txt)

docs_txt[0].metadata
docs_txt[0].page_content[0:100]

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

WEB_URL = "https://news.ifeng.com/c/8Y3TlIcTsj0"
loader_html = WebBaseLoader(WEB_URL)
docs_html = loader_html.load()

len(docs_html)
docs_html[0].metadata
docs_html[0].page_content[:100]

In [ ]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 导入递归字符文本分割器
text_splitter_txt = RecursiveCharacterTextSplitter(chunk_size=384, chunk_overlap=0,
                                                   separators=["\n\n", "\n", " ", "", "。", "，"])
# 导入文本
documents_txt = text_splitter_txt.split_documents(docs_txt)
len(documents_txt)

In [ ]:
documents_txt[0].page_content

___

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_transformer = SentenceTransformer('D:\workspace\project\LLM\models\m3e-base')
sentences = [
    '* Moka 此文本嵌入模型由 MokaAI 训练并开源，训练脚本使用 uniem',
    '* Massive 此文本嵌入模型通过**千万级**的中文句对数据集进行训练',
    '* Mixed 此文本嵌入模型支持中英双语的同质文本相似度计算，异质文本检索等功能，未来还会支持代码检索，ALL in one'
]

#Sentences are encoded by calling model.encode()
emb = sentence_transformer.encode(sentences)

In [ ]:
query1 = "狗"
query2 = "猫"
query3 = "雨"
# 通过对应的 embedding 类生成 query 的 embedding。
emb1 = sentence_transformer.encode(query1)
emb2 = sentence_transformer.encode(query2)
emb3 = sentence_transformer.encode(query3)

In [ ]:
import numpy as np

# 将返回结果转成 numpy 的格式，便于后续计算
emb1 = np.array(emb1)
emb2 = np.array(emb2)
emb3 = np.array(emb3)

np.dot(emb1, emb2)
np.dot(emb3, emb2)
np.dot(emb1, emb3)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer

# sentenceTransformer = SentenceTransformer('D:\workspace\project\LLM\models\m3e-base')
embedding = SentenceTransformerEmbeddings(model_name='D:\workspace\project\LLM\models\m3e-base')


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

WEB_URL = "https://news.ifeng.com/c/8Y3TlIcTsj0"
loader_html = WebBaseLoader(WEB_URL)
docs_html = loader_html.load()

In [ ]:
docs_html

In [ ]:
# persist_directory允许我们将目录保存到磁盘上
# 注意from_documents每次运行都是把数据添加进去
vectordb = Chroma.from_documents(documents=docs_html, embedding=embedding,
                                 persist_directory="D:\workspace\project\LLM\Chroma")

In [ ]:
vectordb.persist()

In [ ]:
vectordb_load = Chroma(
    persist_directory="D:\workspace\project\LLM\Chroma",
    embedding_function=embedding
)

In [ ]:
vectordb_load._collection.count()

In [ ]:
vectordb_load.similarity_search("足协")

#### 检索式问答链

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts.chat import ChatPromptTemplate
from langchain_community.llms import Tongyi

# 创建提示词模板
prompt = ChatPromptTemplate.from_template("""使用下面的语料来回答本模板最末尾的问题。如果你不知道问题的答案，直接回答 "我不知道"，禁止随意编造答案。
        为了保证答案尽可能简洁，你的回答必须不超过三句话，你的回答中不可以带有星号。
        请注意！在每次回答结束之后，你都必须接上 "感谢你的提问" 作为结束语。
        请注意！回答语言与提问语言保持一致。
        以下是一对问题和答案的样例：
            请问：秦始皇的原名是什么
            秦始皇原名嬴政。感谢你的提问。
        
        以下是语料：
<context>
{context}
</context>

Question: {input}""")
#创建检索链
llm = Tongyi(temperature=0.1)
document_chain = create_stuff_documents_chain(llm, prompt)

retriever = vectordb_load.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({
    "input": "王登峰是谁?"
})
response["answer"]

In [ ]:
# 比较：原生模型调用与retrieval_chain调用的区别：
llm.invoke("王登峰是谁?")

#### 检索式对话模型

In [1]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.llms import Tongyi
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

llm = Tongyi()

embedding = SentenceTransformerEmbeddings(model_name='D:\workspace\project\LLM\models\m3e-base')
vectordb = Chroma(
    persist_directory="D:\workspace\project\LLM\Chroma",
    embedding_function=embedding
)
retriever = vectordb.as_retriever()

memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

D:\Program\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
D:\Program\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
from langchain.prompts import PromptTemplate

template = "请用中文回答：{question}"
prompt = PromptTemplate(input_variables=["question"],
                        template=template, )

res = qa.invoke(
    {"question": prompt.format(question="王登峰是谁?")}
)
res["answer"]

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


' 王登峰是教育部体育卫生与艺术教育司的原司长，也曾担任足协原副主席。'

In [5]:
from langchain_core.runnables import RunnablePassthrough

chain = (
        {"question": RunnablePassthrough()}
        |{"question": prompt}
        |qa
)

In [6]:
chain.invoke("王登峰是谁?")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


ValidationError: 2 validation errors for HumanMessage
content
  str type expected (type=type_error.str)
content
  value is not a valid list (type=type_error.list)